In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import numpy as np
import cv2
import os
import glob as gb

In [5]:
import tensorflow as tf

In [6]:
import seaborn as sns
import keras

In [7]:
Res = tf.keras.applications.ResNet50() 

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
Res.summary()

In [ ]:
TrainPath = "../input/intel-image-classification/seg_train/seg_train"
TestPath  = "../input/intel-image-classification/seg_test/seg_test"
PredPath = "../input/intel-image-classification/seg_pred/seg_pred"

In [ ]:
for folder in os.listdir(TrainPath):
    files = gb.glob(pathname = str(TrainPath + "/"+ folder + '/*.jpg'))
    print("Found in {}    {} fiels".format(folder , len(files)))

In [ ]:
PredFiles = gb.glob(pathname = str(PredPath + "/*jpg"))
print("ALL FIELS IN PRED FOLDER ARE " , len(files))

In [ ]:
code = {"street": 0 , "forest":1 , "mountain":2 , "buildings":3 , "glacier":4 , "sea":5}

In [ ]:
def GetCode(n):
    for x , y in code.items():
        if n==y:
            return  x

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
size = []
for file in files:
    img = plt.imread(file)
    size.append(img.shape)
pd.Series(size).value_counts()

In [ ]:
s = 150

In [ ]:
X_train = list()
y_train = list()

for folder in os.listdir(TrainPath):
    Files = gb.glob(pathname = str(TrainPath + "/" + folder + "/*jpg"))
    for file in Files:
        img = cv2.imread(file)
        image_array = cv2.resize(img , (s , s))
        X_train.append(list(image_array))
        y_train.append(code[folder])

In [ ]:
np.array(X_train).shape

In [ ]:
plt.figure(figsize = (20 , 20))
for n , i in enumerate(list(np.random.randint(0 , len(X_train) , 42) )):
    plt.subplot(6 , 7 , n+1)
    plt.imshow(X_train[i])
    plt.axis("off")
    plt.title(GetCode(y_train[i]))

In [ ]:
X_test = list()
y_test = list()

for folder in os.listdir(TestPath):
    Files = gb.glob(pathname = str(TestPath + "/" + folder + "/*jpg"))
    for file in Files:
        img = cv2.imread(file)
        image_array = cv2.resize(img , (s , s))
        X_test.append(list(image_array))
        y_test.append(code[folder])

In [ ]:
X_pred = list()

for folder in os.listdir(PredPath):
    Files = gb.glob(pathname = str(PredPath + "/" + folder + "/*jpg"))
    for file in Files:
        img = cv2.imread(file)
        image_array = cv2.resize(img , (s , s))
        X_Pred.append(list(image_array))

In [ ]:
X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)
X_pred  = np.array(X_pred)

In [ ]:
VGG16 = tf.keras.applications.vgg16.VGG16(input_shape = (s , s , 3) ,include_top=False ) 

In [ ]:
VGG16.summary()

In [ ]:
model2 = keras.Sequential([
    VGG16 ,
    keras.layers.Flatten(),
    keras.layers.Dense(units = 256 , activation = "relu") ,
    keras.layers.Dense(units = 128 , activation = "relu") ,
    keras.layers.Dense(units = 6 )
])

In [ ]:
VGG16.trainable  = False 

In [ ]:
X_train , X_test = X_train / 255 , X_test / 255

In [ ]:
model2.compile(optimizer = "adam" , loss =  tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,  metrics = ["accuracy"])

In [ ]:
model2.summary()

In [ ]:
model = model2.fit(X_train ,y_train , epochs =   , verbose = 1 , batch_size = 10 , validation_data=(X_test , y_test))

In [ ]:
import matplotlib.pyplot as pl

In [ ]:
plt.plot(model.history['accuracy'], label='accuracy')
plt.plot(model.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

In [ ]:
model2.evaluate(X_test ,  y_test , verbose= 1)

In [ ]:
model2.save("Things")

In [ ]:
import os

In [ ]:
CWD = '/kaggle/working'